In [1]:
import pandas as pd
dataset = pd.read_excel('IncidentiModificato.xlsx', sheet_name='Foglio1', index_col=0)
dataset.head()

,DATA,SESSO,ANNI,PESO,ALTEZZA,BMI,Mezzo,Testa:Neurocranio,Testa:Splancnocranio,Testa:Telencefalo,...,Scheletro:Rachide-cervicale,Scheletro:Rachide-toracico,Scheletro:Rachide-lombare,Scheletro:Bacino-e-sacro,Scheletro:Complesso-sterno/claveo/costale,Tot Testa,Tot Torace,Tot Addome,Tot Scheletro,Totale
VERBALE,,,,,,,,,,,,,,,,,,,,,
85567,1999-10-29,0,81,84.0,1.75,27.428571,0,1,0,0,...,0,3,0,3,3,2,0,3,9,14
85829,2000-01-14,1,69,69.0,1.62,26.291724,1,4,4,4,...,0,0,0,0,4,20,7,1,4,32
85977,2000-03-10,1,71,67.0,1.55,27.887617,1,2,0,1,...,0,0,0,0,4,6,0,0,4,10
86220,2000-06-14,1,54,60.0,1.59,23.733238,1,4,0,0,...,0,0,0,0,4,5,3,2,4,14
86247,2000-06-22,1,78,69.0,1.67,24.740937,1,2,0,0,...,0,0,0,0,4,2,0,2,4,8


In [2]:
len(dataset)

130

In [3]:
dataset.index.unique()

Index([    85567,     85829,     85977,     86220,     86247,     86421,
           86839,     86876,     86878,     90056,
       ...
          101097,    101180,    101189,    101544,    101583,    101618,
       '101288E', 'E-97586',       'X',      'X1'],
      dtype='object', name='VERBALE', length=130)

In [4]:
def checkDataset(dataset):
    #controllo che i verbali siano valori unici
    if (len(dataset.index) != len(dataset.index.unique())):
        raise Exception('I verbali non possono essere usati come indice')

    #controllo i valori degli anni
    anni = dataset['ANNI']
    for anno in anni:
        if anno < 1 or anno > 95:
            raise Exception(f'Anno inserito non valido {anno}')
    
    #controllo i valori dei pesi
    pesi = dataset['PESO']
    for peso in pesi:
        if peso < 30 or peso > 120:
            raise Exception(f'Peso inserito non valido {peso}')
    
    #controllo i valori dell'altezza
    altezze = dataset['ALTEZZA']
    for altezza in altezze:
        if altezza < 1.00 or altezza > 2.10:
            raise Exception(f'Altezza inserito non valida {altezza}')
    
    #controllo del BMI
    valori_BMI = dataset['BMI']
    for bmi in valori_BMI:
        if bmi < 10.0 or bmi > 50.0:
            raise Exception(f'bmi inserito non valido {bmi}')
            
    #controllo altri valori compresi tra 0 e 4
    dataset_valori_0_4 = dataset['Testa:Neurocranio']
    for valore in dataset_valori_0_4:
        if valore < 0 or valore > 4:
            raise Exception(f'{valore} non compresa tra 0 e 4')
            

    print("Valori del dataset corretti")
    

checkDataset(dataset)

Valori del dataset corretti


In [48]:
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV, cross_val_score
import time

def addestra(model_class, X, y, model_selection_grid, num_fold_grid_search, num_fold_cross_val, scaling=StandardScaler(), dim_reduction=None):
    
    start_time = time.time()
    
    if scaling is not None:
        sc = scaling
        X_std = sc.fit_transform(X)
    
        if dim_reduction is not None: 
            reducer = dim_reduction
            X_std = reducer.fit_transform(X_std)
    #se scaling è None uso X
    else:
        X_std = X
        
    if confusion_matrix:
        plot_confusion_matrix(y_test, y_pred)
        plt.show()

    clf = GridSearchCV(estimator=model_class(), param_grid=model_selection_grid, cv=num_fold_grid_search, iid=True, n_jobs=-1)
    val = cross_val_score(clf, X_std, y, cv=num_fold_cross_val)
    print("--- %s seconds ---" % (time.time() - start_time))
    return val

    #For integer/None inputs, if the estimator is a classifier and y is either binary or multiclass, StratifiedKFold is used. In all other cases, KFold is used

## Iniziamo considerando solo i totali per distretto

In [49]:
X_total = dataset[['SESSO', 'ANNI', 'PESO', 'ALTEZZA', 'Tot Testa', 'Tot Torace', 'Tot Addome', 'Tot Scheletro']]
y_total = dataset['Mezzo']

In [50]:
from sklearn.svm import SVC

c_space = np.logspace(-4, 3, 10)
gamma_space = np.logspace(-4, 3, 10)

model_selection_grid = [
  {'C': c_space, 'kernel': ['linear'], 'gamma': ['auto']},
  {'C': c_space, 'gamma': gamma_space, 'kernel': ['rbf']},
  {'C': c_space, 'gamma': ['auto', 'scale'], 'kernel': ['rbf']},
  {'C': c_space, 'degree': [2, 3, 5, 9], 'kernel': ['poly'], 'gamma': ['auto']},
 ]

np.mean(addestra(SVC, X_total, y_total, model_selection_grid, num_fold_cross_val=9, num_fold_grid_search=9))

NameError: name 'confusion_matrix' is not defined

In [13]:
#Provato uno scaler diverso

from sklearn.svm import SVC
from sklearn.preprocessing import MaxAbsScaler

c_space = np.logspace(-4, 3, 10)
gamma_space = np.logspace(-4, 3, 10)

model_selection_grid = [
  {'C': c_space, 'kernel': ['linear'], 'gamma': ['auto']},
  {'C': c_space, 'gamma': gamma_space, 'kernel': ['rbf']},
  {'C': c_space, 'gamma': ['auto', 'scale'], 'kernel': ['rbf']},
  {'C': c_space, 'degree': [2, 3, 5, 9], 'kernel': ['poly'], 'gamma': ['auto']},
 ]

np.mean(addestra(SVC, X_total, y_total, model_selection_grid, 9, scaling=MaxAbsScaler()))

0.6434676434676435

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model_selection_grid = {'criterion': ['gini', 'entropy'],
                        'max_leaf_nodes': [None, 2, 5, 10, 50, 100],
                        'max_features': [None, 'sqrt', 'log2'],
                        'max_depth': [None, 2, 5, 10]}

np.mean(addestra(DecisionTreeClassifier, X_total, y_total, model_selection_grid, 9))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model_selection_grid = {'n_estimators': [5, 10, 50, 100, 200],
                        'criterion': ['gini', 'entropy'],
                        'max_leaf_nodes': [None, 2, 5, 10, 50, 100],
                        'max_features': [None, 'sqrt', 'log2'],
                        'max_depth': [None, 2, 5, 10]}

np.mean(addestra(RandomForestClassifier, X_total, y_total, model_selection_grid, 9))

In [ ]:
from sklearn.naive_bayes import GaussianNB

model_selection_grid = {}

np.mean(addestra(GaussianNB, X_total, y_total, model_selection_grid, 9))

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

model_selection_grid = {}

np.mean(addestra(LinearDiscriminantAnalysis, X_total, y_total, model_selection_grid, 9))

In [ ]:
from sklearn.neural_network import MLPClassifier

model_selection_grid = {'max_iter': [5000],
                        'hidden_layer_sizes': [[2], [4], [6], [10], [20], [4, 4], [10, 10]],
                        'activation': ['identity', 'logistic', 'tanh', 'relu']}

np.mean(addestra(MLPClassifier, X_total, y_total, model_selection_grid, 9))

## Aumentiamo la precisione, scendendo nei cinque livelli di ogni distretto

In [ ]:
selected_cols = ['Testa:Neurocranio',
                 'Testa:Splancnocranio',
                 'Testa:Telencefalo',
                 'Testa:Cervelletto',
                 'Testa:Tronco-encefalico',
                 'Torace:Polmoni',
                 'Torace:Trachea/bronchi',
                 'Torace:Cuore',
                 'Torace:Aorta-toracica',
                 'Torace:Diaframma',
                 'Addome:Fegato',
                 'Addome:Milza',
                 'Addome:Aorta-addominale',
                 'Addome:Reni',
                 'Addome:Mesentere',
                 'Scheletro:Rachide-cervicale',
                 'Scheletro:Rachide-toracico',
                 'Scheletro:Rachide-lombare',
                 'Scheletro:Bacino-e-sacro',
                 'Scheletro:Complesso-sterno/claveo/costale']

X_details = dataset[selected_cols]
y_details = dataset['Mezzo']

Iniziamo facendo un controllo su quanto sia possibile ridurre la dimensione dei dati

In [ ]:
pca = PCA(n_components = 20)
pca.fit(X_details)

In [ ]:
%matplotlib inline

from matplotlib import pyplot as plt

plt.plot(range(20), pca.explained_variance_ratio_.cumsum())
plt.show()

In [ ]:
pca.explained_variance_ratio_.cumsum()

In [ ]:
pca.explained_variance_ratio_.cumsum()[12]

Scendendo quindi da 20 a 13 feature si mantiene più del 90% della varianza

### Iniziamo comunque con tutte le 20 feature

In [ ]:
c_space = np.logspace(-4, 3, 10)
gamma_space = np.logspace(-4, 3, 10)

model_selection_grid = [
  {'C': c_space, 'kernel': ['linear'], 'gamma': ['auto']},
  {'C': c_space, 'gamma': gamma_space, 'kernel': ['rbf']},
  {'C': c_space, 'gamma': ['auto', 'scale'], 'kernel': ['rbf']},
  {'C': c_space, 'degree': [2, 3, 5, 9], 'kernel': ['poly'], 'gamma': ['auto']},
 ]

np.mean(addestra(SVC, X_details, y_details, model_selection_grid, 9))

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model_selection_grid = {'criterion': ['gini', 'entropy'],
                        'max_leaf_nodes': [None, 2, 5, 10, 50, 100],
                        'max_features': [None, 'sqrt', 'log2'],
                        'max_depth': [None, 2, 5, 10]}

np.mean(addestra(DecisionTreeClassifier, X_details, y_details, model_selection_grid, 9))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model_selection_grid = {'n_estimators': [5, 10, 50, 100, 200],
                        'criterion': ['gini', 'entropy'],
                        'max_leaf_nodes': [None, 2, 5, 10, 50, 100],
                        'max_features': [None, 'sqrt', 'log2'],
                        'max_depth': [None, 2, 5, 10]}

np.mean(addestra(RandomForestClassifier, X_details, y_details, model_selection_grid, 9))

In [ ]:
from sklearn.naive_bayes import GaussianNB

model_selection_grid = {}

np.mean(addestra(GaussianNB, X_details, y_details, model_selection_grid, 9))

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

model_selection_grid = {}

np.mean(addestra(LinearDiscriminantAnalysis, X_details, y_details, model_selection_grid, 9))

In [ ]:
from sklearn.neural_network import MLPClassifier

model_selection_grid = {'max_iter': [5000],
                        'hidden_layer_sizes': [[2], [4], [6], [10], [20], [4, 4], [10, 10]],
                        'activation': ['identity', 'logistic', 'tanh', 'relu']}

np.mean(addestra(MLPClassifier, X_details, y_details, model_selection_grid, 9))

### Scendiamo a 13 feature con PCA

In [ ]:
c_space = np.logspace(-4, 3, 10)
gamma_space = np.logspace(-4, 3, 10)

model_selection_grid = [
  {'C': c_space, 'kernel': ['linear'], 'gamma': ['auto']},
  {'C': c_space, 'gamma': gamma_space, 'kernel': ['rbf']},
  {'C': c_space, 'gamma': ['auto', 'scale'], 'kernel': ['rbf']},
  {'C': c_space, 'degree': [2, 3, 5, 9], 'kernel': ['poly'], 'gamma': ['auto']},
 ]

np.mean(addestra(SVC, X_details, y_details, model_selection_grid, 9, dim_reduction=PCA(n_components=13)))

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model_selection_grid = {'criterion': ['gini', 'entropy'],
                        'max_leaf_nodes': [None, 2, 5, 10, 50, 100],
                        'max_features': [None, 'sqrt', 'log2'],
                        'max_depth': [None, 2, 5, 10]}

np.mean(addestra(DecisionTreeClassifier, X_details, y_details, model_selection_grid, 9, dim_reduction=PCA(n_components=13)))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model_selection_grid = {'n_estimators': [5, 10, 50, 100, 200],
                        'criterion': ['gini', 'entropy'],
                        'max_leaf_nodes': [None, 2, 5, 10, 50, 100],
                        'max_features': [None, 'sqrt', 'log2'],
                        'max_depth': [None, 2, 5, 10]}

np.mean(addestra(RandomForestClassifier, X_details, y_details, model_selection_grid, 9, dim_reduction=PCA(n_components=13)))

In [ ]:
from sklearn.naive_bayes import GaussianNB

model_selection_grid = {}

np.mean(addestra(GaussianNB, X_details, y_details, model_selection_grid, 9, dim_reduction=PCA(n_components=13)))

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

model_selection_grid = {}

np.mean(addestra(LinearDiscriminantAnalysis, X_details, y_details, model_selection_grid, 9, dim_reduction=PCA(n_components=13)))

In [ ]:
from sklearn.neural_network import MLPClassifier

model_selection_grid = {'max_iter': [5000],
                        'hidden_layer_sizes': [[2], [4], [6], [10], [20], [4, 4], [10, 10]],
                        'activation': ['identity', 'logistic', 'tanh', 'relu']}

np.mean(addestra(MLPClassifier, X_details, y_details, model_selection_grid, 9, dim_reduction=PCA(n_components=13)))

### Proviamo a ridurre le componenti a 10 con t-SNE

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
from sklearn.svm import SVC

c_space = np.logspace(-4, 3, 10)
gamma_space = np.logspace(-4, 3, 10)

model_selection_grid = [
  {'C': c_space, 'kernel': ['linear'], 'gamma': ['auto']},
  {'C': c_space, 'gamma': gamma_space, 'kernel': ['rbf']},
  {'C': c_space, 'gamma': ['auto', 'scale'], 'kernel': ['rbf']},
  {'C': c_space, 'degree': [2, 3, 5, 9], 'kernel': ['poly'], 'gamma': ['auto']},
 ]

np.mean(addestra(SVC, X_details, y_details, model_selection_grid, 9, dim_reduction=TSNE(n_components=10, method='exact')))

In [ ]:
from sklearn.tree import DecisionTreeClassifier

model_selection_grid = {'criterion': ['gini', 'entropy'],
                        'max_leaf_nodes': [None, 2, 5, 10, 50, 100],
                        'max_features': [None, 'sqrt', 'log2'],
                        'max_depth': [None, 2, 5, 10]}

np.mean(addestra(DecisionTreeClassifier, X_details, y_details, model_selection_grid, 9, dim_reduction=TSNE(n_components=3)))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

model_selection_grid = {'n_estimators': [5, 10, 50, 100, 200],
                        'criterion': ['gini', 'entropy'],
                        'max_leaf_nodes': [None, 2, 5, 10, 50, 100],
                        'max_features': [None, 'sqrt', 'log2'],
                        'max_depth': [None, 2, 5, 10]}

np.mean(addestra(RandomForestClassifier, X_details, y_details, model_selection_grid, 9, dim_reduction=TSNE(n_components=3)))

In [ ]:
from sklearn.naive_bayes import GaussianNB

model_selection_grid = {}

np.mean(addestra(GaussianNB, X_details, y_details, model_selection_grid, 9, dim_reduction=TSNE(n_components=3)))

In [ ]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

model_selection_grid = {}

np.mean(addestra(LinearDiscriminantAnalysis, X_details, y_details, model_selection_grid, 9, dim_reduction=TSNE(n_components=3)))

In [ ]:
from sklearn.neural_network import MLPClassifier

model_selection_grid = {'max_iter': [5000],
                        'hidden_layer_sizes': [[2], [4], [6], [10], [20], [4, 4], [10, 10]],
                        'activation': ['identity', 'logistic', 'tanh', 'relu']}

np.mean(addestra(MLPClassifier, X_details, y_details, model_selection_grid, 9, dim_reduction=TSNE(n_components=3)))